# Google Code Jam 
## Kickstart Round G 2018

By Winn Chow, winnchow@gmail.com

A. Product Triplets

https://code.google.com/codejam/contest/5374486/dashboard#s=p0

### Prepare Test Cases - Solve A-small

In [1]:
filename_s = "A-small-practice.in"

In [2]:
def read_data_file(filename):
    """Read input file and return test cases in a list."""
    
    with open(filename, "r") as f:
        # Read number of test cases
        n = int(f.readline())
        print("n = " + str(n))
        # Read all the rest
        input_cases = f.readlines()

    # Read each test case
    def read_case(j):
        # 1st line - total number of numbers
        tmp = input_cases[j].split()
        m = int(tmp[0])
        # 2nd line - the numbers
        tmp = list(map(int, input_cases[j + 1].split()))
        return tmp

    # Store all the test cases
    test_cases = []
    for i in range(0, n*2, 2):
        test_cases.append(read_case(i))

    print("Total number of test cases:", len(test_cases))
    
    return test_cases

In [3]:
test_cases_s = read_data_file(filename_s)

n = 30
Total number of test cases: 30


### Solution

We first sort the numbers in decreasing order (but 0s have to be in the front) and then compare each tuple of 3 numbers. We stop when the numbers get too small. 

In [50]:
import math
from tqdm import *

def solve_small(test_cases):
    """Return the result to small test cases"""
    
    # Each case
    result = []
    for l in tqdm(test_cases):
        # Count
        count = 0
        # Sort it first, as 0 * any number == 0, we have to put zeros in the front
        l.sort(key=lambda x: -x if x != 0 else -math.inf)
        #print(l)
        # For each x
        for i, x in enumerate(l):
            # if we are not at the end
            #print(">>>>>>>>x", x)
            if len(l) - i >= 3:
                # Check if x == y * z
                # For each y
                for j, y in enumerate(l[i+1:-1]):
                    #print("y", y)
                    # For each z
                    for z in l[i+1+j+1:]:
                        #print("z", z)
                        if x == y * z:
                            count += 1
                            #print("Yes")
                        # All numbers are stored in decreasing order, stop once it gets too small
                        elif x > y * z:
                            #print("No")
                            break
        result.append(count)
    print("Done")
    return result

In [51]:
result_s = solve_small(test_cases_s)
print(result_s)

100%|██████████| 30/30 [00:05<00:00,  4.85it/s]


Done
[1, 6, 1, 0, 0, 64, 799224, 0, 785997, 1313400, 49, 0, 0, 1313400, 0, 0, 178, 0, 656700, 118, 142, 91, 0, 0, 192, 0, 0, 656700, 77, 0]


### Prepare output

In [59]:
def write_output_file(result, filename):
    """Write result to an output file"""
    
    with open(filename, "w") as f:
        for i, r in enumerate(result):
            f.write("Case #" + str(i + 1) + ": " + str(r) + "\n")

    print("Done")

In [ ]:
write_output_file(result_s, "A-small-practice.out")

### Prepare Test Cases - Solve A-large

In [55]:
filename_l = "A-large-practice.in"

In [56]:
test_cases_l= read_data_file(filename_l)

n = 30
Total number of test cases: 30


### Solution

We first sort the numbers in decreasing order (but 0s have to be in the front).

For the large dataset, we need to speed up by:

1. The first 3 repeated values of a number can form a unique tuple e.g. (1, 1, 1). But further repeated values will not create more unique tupbles. They just form more combinations of the same tuples. We can use nCr to work out the number of additional combinations.
2. If x is not divisible by y, continue to the next y.
3. Use binary search to find z for each pair of x and y. The target z is x divided by y.

In [52]:
import math
from scipy.special import comb
from collections import Counter
from bisect import bisect_left 
from tqdm import *

def binary_search(a, x): 
    """ Do binary search and return -1 if not found
        Reference: https://stackoverflow.com/questions/212358/binary-search-bisection-in-python """
    
    i = bisect_left(a, x) 
    if i != len(a) and a[i] == x: 
        return i 
    else: 
        return -1

def solve_large(test_cases):
    """Return the result to large test cases"""
    
    # Each case
    result = []
    for l in tqdm(test_cases):
        # Count
        count = 0
        
        # Do counting
        lc = Counter(l)
        #print(lc)
        new_l = []
        
        # Build a list with a max of 3 repeated values
        # All other repeated values are only combinations, we can use nCr to work them out at the end
        for e, c in lc.items():
            new_l += [e] * min(c, 3)
        #print(new_l)
        
        # Sort it first, as 0 * any number == 0, we have to put zeros in the front
        new_l.sort(key=lambda x: -x if x != 0 else -math.inf)
        
        # Store all the result tuples
        dict_tuples = {}
        # For each x
        for i, x in enumerate(new_l):
            # if we are not at the end
            #print(">>>>>>>>x", x)
            if len(new_l) - i >= 3:
                # Check if x == y * z
                # For each y
                for j, y in enumerate(new_l[i+1:-1]):
                    #print("y", y)
                    # If it is not divisible by y
                    if y > 0 and x % y != 0:
                        continue
                    
                    # We look for target z 
                    if y > 0:
                        target_z = x // y
                        #print("target_z = " + str(target_z))
                        #print(new_l[:i+1+j:-1])
                        
                        # Use binary search to find the target z, but need to reverse the list 
                        # so that it is in increasing order
                        z = binary_search(new_l[:i+1+j:-1], target_z)
                        #print("found = " + str(z))
                        if z != -1:
                            dict_tuples[(x, y, target_z)] = True
                    # if both x and y are zero, all z are answers
                    elif x == 0 and y == 0:
                        for z in new_l[i+1+j+1:]:
                            dict_tuples[(x, y, z)] = True
        
        # Calculate all the combinations for the repeated values, nCr
        for t in dict_tuples.keys():
            ct = Counter(t)
            tmp_count = 1
            for e, c in ct.items():
                # nCr
                tmp_count *= comb(lc[e], c, exact=True)
            count += tmp_count
        result.append(count)
    print("Done")
    return result

In [57]:
result_l = solve_large(test_cases_l)
print(result_l)

100%|██████████| 30/30 [01:03<00:00,  3.20s/it]


Done
[1, 6, 1, 0, 26, 24, 26, 21431968, 26189, 54, 0, 79, 0, 0, 26, 57142169000, 11, 25535, 63500, 35540676584, 28571084500, 20877777, 35686322024, 15, 57142169000, 28571084500, 27, 20368986, 21085, 25881]


### Prepare output

In [61]:
write_output_file(result_l, "A-large-practice.out")

Done
